In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Output')
cur_date = "042322"

library(readxl)
library(tidyverse)
library(reshape2)


#reading in file
proteomics_df = data.frame(read_excel("041822_Imputed_Proteomics_Data.xlsx"))
protein_info_df = data.frame(read_excel("List of Proteins Detected_041922.xlsx")) %>%
    rename(Protein = Accession)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(proteomics_df)
head(protein_info_df)

,ID,Set,Treatment,Protein,Value
,<chr>,<chr>,<chr>,<chr>,<dbl>
1,CoCulture_Exposed1,Set1,CoCulture_Exposed,P19012,476617.6
2,CoCulture_Exposed1,Set1,CoCulture_Exposed,P19013,1077007.5
3,CoCulture_Exposed1,Set1,CoCulture_Exposed,O43897,2245997.6
4,CoCulture_Exposed1,Set1,CoCulture_Exposed,P20700,2388716.6
5,CoCulture_Exposed1,Set1,CoCulture_Exposed,P22102,1790626.6
6,CoCulture_Exposed1,Set1,CoCulture_Exposed,O75367,874143.2


,Protein,Gene.name,Description,GOCC.name,Contaminant
,<chr>,<chr>,<chr>,<chr>,<chr>
1,P02751,FN1,Fibronectin OS=Homo sapiens OX=9606 GN=FN1 PE=1 SV=5,apical plasma membrane;basal lamina;blood microparticle;cell part;cytoplasmic membrane-bounded vesicle lumen;cytoplasmic part;cytoplasmic vesicle part;endoplasmic reticulum-Golgi intermediate compartment;extracellular matrix;extracellular matrix part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;fibrinogen complex;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle part;intracellular part;macromolecular complex;membrane part;membrane-bounded organelle;membrane-bounded vesicle;membrane-enclosed lumen;organelle;organelle lumen;organelle part;plasma membrane part;platelet alpha granule lumen;protein complex;secretory granule lumen;vesicle;vesicle lumen,-
2,P02769,SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor,SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor,NA,+
3,P07996,THBS1,Thrombospondin-1 OS=Homo sapiens OX=9606 GN=THBS1 PE=1 SV=2,cell part;cell surface;cytoplasmic membrane-bounded vesicle;cytoplasmic membrane-bounded vesicle lumen;cytoplasmic part;cytoplasmic vesicle;cytoplasmic vesicle part;endoplasmic reticulum;endoplasmic reticulum lumen;endoplasmic reticulum part;external side of plasma membrane;extracellular matrix;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;fibrinogen complex;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle lumen;intracellular organelle part;intracellular part;macromolecular complex;membrane part;membrane-bounded organelle;membrane-bounded vesicle;membrane-enclosed lumen;organelle;organelle lumen;organelle part;plasma membrane part;platelet alpha granule;platelet alpha granule lumen;protein complex;sarcoplasmic reticulum;secretory granule lumen;stored secretory granule;vesicle;vesicle lumen,-
4,Q2UVX4,SWISS-PROT:Q2UVX4 (Bos taurus) Complement C3 precursor,SWISS-PROT:Q2UVX4 (Bos taurus) Complement C3 precursor,NA,+
5,P98160,HSPG2,Basement membrane-specific heparan sulfate proteoglycan core protein OS=Homo sapiens OX=9606 GN=HSPG2 PE=1 SV=4,adherens junction;anchoring junction;basal lamina;cell junction;cell part;cell-substrate adherens junction;cell-substrate junction;cytoplasmic part;extracellular matrix;extracellular matrix part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;focal adhesion;Golgi apparatus part;Golgi lumen;intracellular organelle lumen;intracellular organelle part;intracellular part;lysosomal lumen;membrane;membrane-bounded organelle;membrane-bounded vesicle;membrane-enclosed lumen;organelle;organelle lumen;organelle part;plasma membrane;vacuolar lumen;vacuolar part;vesicle,-
6,P12763,SWISS-PROT:P12763 (Bos taurus) Alpha-2-HS-glycoprotein precursor,SWISS-PROT:P12763 (Bos taurus) Alpha-2-HS-glycoprotein precursor,NA,+


In [3]:
proteomics_df = proteomics_df %>%
    #just want a column that specifies the treatment and exposure separately
    separate(Treatment, c("Treatment", "Exposure"), sep = "_") 

head(proteomics_df)

,ID,Set,Treatment,Exposure,Protein,Value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,CoCulture_Exposed1,Set1,CoCulture,Exposed,P19012,476617.6
2,CoCulture_Exposed1,Set1,CoCulture,Exposed,P19013,1077007.5
3,CoCulture_Exposed1,Set1,CoCulture,Exposed,O43897,2245997.6
4,CoCulture_Exposed1,Set1,CoCulture,Exposed,P20700,2388716.6
5,CoCulture_Exposed1,Set1,CoCulture,Exposed,P22102,1790626.6
6,CoCulture_Exposed1,Set1,CoCulture,Exposed,O75367,874143.2


In [4]:
#explain why I split this
proteomics_df_split = proteomics_df %>%
    group_by(Set) %>%
    group_split()

set1_proteomics_df = proteomics_df_split[[1]]
set2_proteomics_df = proteomics_df_split[[2]]

head(set1_proteomics_df)

ID,Set,Treatment,Exposure,Protein,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
CoCulture_Exposed1,Set1,CoCulture,Exposed,P19012,476617.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,P19013,1077007.5
CoCulture_Exposed1,Set1,CoCulture,Exposed,O43897,2245997.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,P20700,2388716.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,P22102,1790626.6
CoCulture_Exposed1,Set1,CoCulture,Exposed,O75367,874143.2


Running t tests to determine if there are statistically significant distribution differences in co-culture unexposed vs. exposed and tri-culture unexposed vs. exposed. 

In [5]:
t_test_values = function(df, set_id){
    #"""
    #Running t tests after filtering for set, treatment, exposure, and protein using a loop. 
    #Ultimately using this test to compare proteins (unexposed vs. exposed).

    #:param: subsetted dataframe, empty dataframe
    #:output: a dataframe containing the set, treatment, exposure, protein, u stat, p value, p adj

    #"""
    treatments = unique(df$Treatment)
    proteins = unique(df$Protein)
    
    values_df = data.frame()
    #iterating through each tx, protein

    for(i in 1:length(treatments)){
        for(j in 1:length(proteins)){
            #unexposed df
            unexposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Vehicle", Protein == proteins[j])
            #exposed df
            exposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Exposed", Protein == proteins[j])

            #t test
            t_test = t.test(unexposed_df$Value, exposed_df$Value)
            
            #calculating FC to get directionality
            FC = log2(mean(unexposed_df$Value)/mean(exposed_df$Value))

            #contains smoking status compared, compartment, cytokine, u stat, and p value
            values_vector = cbind(set_id, treatments[i], proteins[j], FC, t_test$statistic, t_test$p.value)
            values_df = rbind(values_df, values_vector)
            }
        }

    
    #adding col names
    colnames(values_df) = c("Set", "Treatment", "Protein", "log2FC", "Statistic", "P Value")
    
    
    #calculating padj values
    padj_vector = c()
    #adjusting within each treatment
    for (i in 1:length(treatments)){
        `P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")
        padj_vector = c(padj_vector, `P Adj`)
    }
    
    values_df = cbind(values_df, padj_vector) %>%
        rename(`P Adj` = padj_vector)

    values_df$log2FC = as.numeric(values_df$log2FC) 
    values_df$Statistic = as.numeric(values_df$Statistic)
    values_df$`P Value` = as.numeric(values_df$`P Value`) 
    
    return(values_df)
}

In [6]:
# calling fn
set1_t_test = t_test_values(set1_proteomics_df, "Set1")
set2_t_test = t_test_values(set2_proteomics_df, "Set2")

In [7]:
# creating 1 df
t_test_df = rbind(set1_t_test, set2_t_test)
head(t_test_df)

,Set,Treatment,Protein,log2FC,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
t,Set1,CoCulture,P19012,0.3539417,0.6033149,0.5672249,0.8207093
t1,Set1,CoCulture,P19013,-0.1250127,-0.1394369,0.8938424,0.9672176
t2,Set1,CoCulture,O43897,-0.3537176,-1.2881706,0.2415972,0.7149305
t3,Set1,CoCulture,P20700,-0.1311273,-0.4011018,0.6984561,0.8649296
t4,Set1,CoCulture,P22102,0.8732620,1.0907687,0.3118780,0.7329387
t5,Set1,CoCulture,O75367,-0.3787917,-0.6036463,0.5633036,0.8186874


In [8]:
# exporting t test results
write.csv(t_test_df, paste0(Output,"/", cur_date, "_Statistical_Results_Only.csv"), row.names = FALSE)

In [9]:
t_test_df = pivot_longer(t_test_df, cols = 4:7, names_to = "var", values_to = "Value") %>%
    unite("Variable", Treatment,var) %>%
    select(-Set)

t_test_df = t_test_df[,c(2,1,3)]

head(t_test_df)

Protein,Variable,Value
<chr>,<chr>,<dbl>
P19012,CoCulture_log2FC,0.3539417
P19012,CoCulture_Statistic,0.6033149
P19012,CoCulture_P Value,0.5672249
P19012,CoCulture_P Adj,0.8207093
P19013,CoCulture_log2FC,-0.1250127
P19013,CoCulture_Statistic,-0.1394369


In [10]:
#calculating overall avg. abundance and within smoking groups
avg_abundance_df = proteomics_df %>%
    group_by(Protein) %>%
    summarize(Avg_Abundance = mean(Value))

#stratified by treatment
avg_abundance_df_stratified = proteomics_df %>%
    group_by(Treatment, Protein) %>%
    summarize(Avg_Abundance_stratified = mean(Value))

head(avg_abundance_df)

`summarise()` has grouped output by 'Treatment'. You can override using the
`.groups` argument.


Protein,Avg_Abundance
<chr>,<dbl>
A0A096LP49,51897126
A0A2R8Y4Y8,1745284
A0AVT1,10826356
A1L4H1,120968155
A2I7N0,44219136
A2I7N1,8240726


In [11]:
#changing the stratified dfs to making combining easier
#first splitting dfs
split_avg_abundance_stratified = avg_abundance_df_stratified %>%
    group_by(Treatment) %>%
    group_split()

#adding treatment to the title
coculture_exposed_avg_abundance = split_avg_abundance_stratified[[1]] %>%
    select(-Treatment) %>%
    rename(Coculture_Exposed_Avg_Abundance = Avg_Abundance_stratified)
coculture_vehicle_avg_abundance = split_avg_abundance_stratified[[2]] %>%
    select(-Treatment) %>%
    rename(Coculture_Vehicle_Avg_Abundance = Avg_Abundance_stratified)
triculture_exposed_avg_abundance = split_avg_abundance_stratified[[1]] %>%
    select(-Treatment) %>%
    rename(Triculture_Exposed_Avg_Abundance = Avg_Abundance_stratified)
triculture_vehicle_avg_abundance = split_avg_abundance_stratified[[2]] %>%
    select(-Treatment) %>%
    rename(Triculture_Vehicle_Avg_Abundance = Avg_Abundance_stratified)

head(coculture_exposed_avg_abundance)

Protein,Coculture_Exposed_Avg_Abundance
<chr>,<dbl>
A0A096LP49,51897126
A0A2R8Y4Y8,1745284
A1L4H1,19473534
A2I7N0,62531054
A2I7N1,12761148
A2I7N3,71043165


In [12]:
# combining into 1 abundance df
stratified_abundance_df = inner_join(inner_join(coculture_exposed_avg_abundance, coculture_vehicle_avg_abundance), 
      inner_join(triculture_exposed_avg_abundance, triculture_vehicle_avg_abundance))
abundance_df = inner_join(avg_abundance_df, stratified_abundance_df)

head(abundance_df)

Joining, by = "Protein"
Joining, by = "Protein"
Joining, by = "Protein"
Joining, by = "Protein"


Protein,Avg_Abundance,Coculture_Exposed_Avg_Abundance,Coculture_Vehicle_Avg_Abundance,Triculture_Exposed_Avg_Abundance,Triculture_Vehicle_Avg_Abundance
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A1L4H1,120968155,19473534,222462776,19473534,222462776
A2I7N0,44219136,62531054,25907218,62531054,25907218
A2I7N1,8240726,12761148,3720303,12761148,3720303
A2I7N3,59870079,71043165,48696993,71043165,48696993
B9A064,72000254,54532126,89468383,54532126,89468383
ENSEMBL.ENSBTAP00000006074,592952168,744021707,441882630,744021707,441882630


In [13]:
# melting df so combining with other data frames is easier/ more clear
abundance_df = melt(abundance_df, variable.name = "Variable", value.name = "Value")
head(abundance_df)

Using Protein as id variables



,Protein,Variable,Value
,<chr>,<fct>,<dbl>
1,A1L4H1,Avg_Abundance,120968155
2,A2I7N0,Avg_Abundance,44219136
3,A2I7N1,Avg_Abundance,8240726
4,A2I7N3,Avg_Abundance,59870079
5,B9A064,Avg_Abundance,72000254
6,ENSEMBL.ENSBTAP00000006074,Avg_Abundance,592952168


In [14]:
# combining into 1 df
final_df = rbind(t_test_df, abundance_df)
final_df = full_join(final_df, protein_info_df)
head(final_df)

Joining, by = "Protein"


Protein,Variable,Value,Gene.name,Description,GOCC.name,Contaminant
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
P19012,CoCulture_log2FC,0.3539417,KRT15,"Keratin, type I cytoskeletal 15 OS=Homo sapiens OX=9606 GN=KRT15 PE=1 SV=3",cell part;cytoskeletal part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region part;extracellular vesicular exosome;intermediate filament;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle part;intracellular part;macromolecular complex;membrane-bounded organelle;membrane-bounded vesicle;nucleus;organelle;organelle part;protein complex;vesicle,-
P19012,CoCulture_Statistic,0.6033149,KRT15,"Keratin, type I cytoskeletal 15 OS=Homo sapiens OX=9606 GN=KRT15 PE=1 SV=3",cell part;cytoskeletal part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region part;extracellular vesicular exosome;intermediate filament;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle part;intracellular part;macromolecular complex;membrane-bounded organelle;membrane-bounded vesicle;nucleus;organelle;organelle part;protein complex;vesicle,-
P19012,CoCulture_P Value,0.5672249,KRT15,"Keratin, type I cytoskeletal 15 OS=Homo sapiens OX=9606 GN=KRT15 PE=1 SV=3",cell part;cytoskeletal part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region part;extracellular vesicular exosome;intermediate filament;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle part;intracellular part;macromolecular complex;membrane-bounded organelle;membrane-bounded vesicle;nucleus;organelle;organelle part;protein complex;vesicle,-
P19012,CoCulture_P Adj,0.8207093,KRT15,"Keratin, type I cytoskeletal 15 OS=Homo sapiens OX=9606 GN=KRT15 PE=1 SV=3",cell part;cytoskeletal part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region part;extracellular vesicular exosome;intermediate filament;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle part;intracellular part;macromolecular complex;membrane-bounded organelle;membrane-bounded vesicle;nucleus;organelle;organelle part;protein complex;vesicle,-
P19013,CoCulture_log2FC,-0.1250127,SWISS-PROT:P19013 Tax_Id=9606 Gene_Symbol=KRT4 keratin 4,SWISS-PROT:P19013 Tax_Id=9606 Gene_Symbol=KRT4 keratin 4,cell part;cell surface;cytoskeletal part;cytoskeleton;intermediate filament;intermediate filament cytoskeleton;intracellular membrane-bounded organelle;intracellular non-membrane-bounded organelle;intracellular organelle;intracellular organelle part;intracellular part;keratin filament;macromolecular complex;membrane-bounded organelle;non-membrane-bounded organelle;nucleus;organelle;organelle part;protein complex,+
P19013,CoCulture_Statistic,-0.1394369,SWISS-PROT:P19013 Tax_Id=9606 Gene_Symbol=KRT4 keratin 4,SWISS-PROT:P19013 Tax_Id=9606 Gene_Symbol=KRT4 keratin 4,cell part;cell surface;cytoskeletal part;cytoskeleton;intermediate filament;intermediate filament cytoskeleton;intracellular membrane-bounded organelle;intracellular non-membrane-bounded organelle;intracellular organelle;intracellular organelle part;intracellular part;keratin filament;macromolecular complex;membrane-bounded organelle;non-membrane-bounded organelle;nucleus;organelle;organelle part;protein complex,+


In [15]:
# exporting
write.csv(final_df, paste0(Output,"/", cur_date, "_Distribution_Analysis_All_Data.csv"), row.names = FALSE)